In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta

# Даты
start_date = datetime(2025, 4, 13)
end_date = datetime.now()

# Результаты
data = []

# Заголовки
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/91.0.4472.124"
}

current_date = start_date
last_year = None

while current_date <= end_date:
    date_str = current_date.strftime("%d.%m.%Y")
    year_now = current_date.year
    if year_now != last_year:
        print(f"📅 Начался парсинг {year_now} года...")
        last_year = year_now

    url = f"https://nbt.tj/tj/kurs/gold.php?date={date_str}"
    price = None

    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            tables = soup.find_all("table")

            # Поиск таблицы, содержащей "грамм" и "цена"
            for table in tables:
                rows = table.find_all("tr")
                for row in rows:
                    cols = row.find_all("td")
                    if len(cols) >= 3:
                        weight = cols[0].text.strip().lower().replace(" ", "")
                        if weight.startswith("5"):
                            price = cols[2].text.strip()  # Цена продажи
                            break
                if price:
                    break
    except Exception as e:
        pass  # Игнорируем ошибки, продолжаем

    data.append({
        "date": date_str,
        "price_sell_5g": price
    })
    current_date += timedelta(days=1)

# Сохраняем в Excel
df = pd.DataFrame(data)
df.to_excel("Парсинг_остальных_данных.xlsx", index=False)
print("✅ Данные сохранены в Парсинг_остальных_данных.xlsx")


📅 Начался парсинг 2025 года...
✅ Данные сохранены в Парсинг_остальных_данных.xlsx


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   date           26 non-null     object
 1   price_sell_5g  26 non-null     object
dtypes: object(2)
memory usage: 548.0+ bytes


In [5]:
df.isna().sum()

date             0
price_sell_5g    0
dtype: int64

In [7]:
df.shape

(26, 2)

In [9]:
df

,date,price_sell_5g
0,13.04.2025,5829.06
1,14.04.2025,5829.06
2,15.04.2025,5836.56
3,16.04.2025,5842.28
4,17.04.2025,5969.17
5,18.04.2025,5979.95
6,19.04.2025,5924.52
7,20.04.2025,5924.52
8,21.04.2025,5924.52
9,22.04.2025,5909.41
